In [36]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import lxml

import re

print('Libraries imported.')

Libraries imported.


In [16]:
dict_link = {
    'Houston': 'https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods',
    'Chicago': 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
}

dict_cities = {
    
}

In [26]:
def get_coord(address):

    geolocator = Nominatim(user_agent="to_explorer")
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print(f'The geograpical coordinate of {address} are {latitude:.2f}, {longitude:.2f}.')
        return latitude, longitude
    except:
        return None, None

## Download Data

### New York

In [17]:
with open('../data/newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

dict_cities['New York'] = neighborhoods

### Houston

In [18]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_Houston_neighborhoods', header=0)[0]

In [19]:
table.head()

,#,Name,Location relative to Downtown Houston,Approximate boundaries
0,1,Willowbrook,Northwest,Along Texas State Highway 249 northwest of Bel...
1,2,Greater Greenspoint,North,Around the junction of Beltway 8 and Interstat...
2,3,Carverdale,Northwest,South of the junction of Beltway 8 and U.S. Ro...
3,4,Fairbanks / Northwest Crossing,Northwest,Along U.S. Route 290 between Interstate 610 an...
4,5,Greater Inwood,Northwest,North of Fairbanks / Northwest Crossing and ea...


In [40]:
neighborhoods = pd.Series(re.split(',; | / ', ',; '.join(table.rename(columns={'Name': 'Neighborhood'})['Neighborhood'].tolist())), name='Neighborhood').to_frame()

In [42]:
neighborhoods[['Latitude', 'Longitude']] = pd.DataFrame((neighborhoods.Neighborhood + ', Houston, USA').apply(get_coord).tolist(), index=neighborhoods.index)

The geograpical coordinate of Willowbrook, Houston, USA are 29.66, -95.46.
The geograpical coordinate of Greater Greenspoint, Houston, USA are 29.94, -95.42.
The geograpical coordinate of Carverdale, Houston, USA are 29.85, -95.54.
The geograpical coordinate of Fairbanks, Houston, USA are 29.85, -95.52.
The geograpical coordinate of Northwest Crossing, Houston, USA are 29.85, -95.50.
The geograpical coordinate of Acres Home, Houston, USA are 29.77, -95.33.
The geograpical coordinate of Hidden Valley, Houston, USA are 33.59, -84.19.
The geograpical coordinate of Westbranch, Houston, USA are 29.84, -95.55.
The geograpical coordinate of Addicks, Houston, USA are 29.78, -95.64.
The geograpical coordinate of Spring Branch West, Houston, USA are 29.80, -95.55.
The geograpical coordinate of Langwood, Houston, USA are 29.83, -95.48.
The geograpical coordinate of Independence Heights, Houston, USA are 29.84, -95.40.
The geograpical coordinate of Lazybrook, Houston, USA are 29.80, -95.44.
The ge

In [43]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Willowbrook,29.660254,-95.456096
1,Greater Greenspoint,29.944719,-95.416074
2,Carverdale,29.848687,-95.539450
3,Fairbanks,29.852726,-95.524386
4,Northwest Crossing,29.853820,-95.504597


In [44]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0]

,Neighborhood,Latitude,Longitude
5,Greater Inwood,NaN,NaN
10,Park Ten,NaN,NaN
13,Central Northwest (formerly Near Northwest),NaN,NaN
17,Greater Heights,NaN,NaN
18,Memorial,NaN,NaN
23,Mid-West (formerly Woodlake/Briarmeadow),NaN,NaN
25,Washington Avenue Coalition,NaN,NaN
29,Neartown,NaN,NaN
41,South Main,NaN,NaN
42,Brays Oaks (formerly Greater Fondren S.W.),NaN,NaN


Park Ten is the same as Addicks

In [45]:
neighborhoods = neighborhoods[neighborhoods.Neighborhood != 'Park Ten']

In [46]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0]

,Neighborhood,Latitude,Longitude
5,Greater Inwood,NaN,NaN
13,Central Northwest (formerly Near Northwest),NaN,NaN
17,Greater Heights,NaN,NaN
18,Memorial,NaN,NaN
23,Mid-West (formerly Woodlake/Briarmeadow),NaN,NaN
25,Washington Avenue Coalition,NaN,NaN
29,Neartown,NaN,NaN
41,South Main,NaN,NaN
42,Brays Oaks (formerly Greater Fondren S.W.),NaN,NaN
67,Greater Fifth Ward,NaN,NaN


Data acquired from Google Map

In [47]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0, ['Latitude', 'Longitude']] = [[29.8800973,-95.5142869],
[29.8295093,-95.4602677],
[29.7953545,-95.4219342],
[29.7653329,-95.6280326],
[29.7361698,-95.5255453],
[29.7741598,-95.4281763],
[29.7469666,-95.413642],
[29.6752812,-95.4347469],
[29.653808,-95.5513956],
[29.7754873,-95.3459998],
[29.7506196,-95.2825196],
[29.7247024,-95.3918737],
[29.6952297,-95.3625073],
[29.6262917,-95.2876597],
[29.6039379,-95.2377624]]

In [49]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0]

,Neighborhood,Latitude,Longitude


In [51]:
neighborhoods.loc[(neighborhoods.Latitude - 29).abs() > 1]

,Neighborhood,Latitude,Longitude
7,Hidden Valley,33.594555,-84.191030
21,Briar Forest,32.995124,-97.542801
30,Montrose,39.164153,-88.381063
32,Sharpstown,39.136499,-76.218564
33,Gulfton,37.183669,-94.544951
34,University Place,37.936437,-88.609499
35,Westwood,31.085472,-94.158249
36,Braeburn,37.264581,-80.026706
43,Westbury,30.102158,-94.309631
47,Central Southwest,35.803687,-89.537297


Homestead is the same as East Little York;<br>
Gulfgate Riverview is the same as Pine Valley<br>
Lawndale is the same as Wayside

In [56]:
neighborhoods = neighborhoods[~neighborhoods.Neighborhood.isin(('Homestead', 'Gulfgate Riverview', 'Lawndale'))]

In [57]:
neighborhoods.loc[(neighborhoods.Latitude - 29).abs() > 1]

,Neighborhood,Latitude,Longitude
7,Hidden Valley,33.594555,-84.191030
21,Briar Forest,32.995124,-97.542801
30,Montrose,39.164153,-88.381063
32,Sharpstown,39.136499,-76.218564
33,Gulfton,37.183669,-94.544951
34,University Place,37.936437,-88.609499
35,Westwood,31.085472,-94.158249
36,Braeburn,37.264581,-80.026706
43,Westbury,30.102158,-94.309631
47,Central Southwest,35.803687,-89.537297


In [58]:
neighborhoods.loc[(neighborhoods.Latitude - 29).abs() > 1, ['Latitude', 'Longitude']] = [[29.8901821,-95.4291368],
[29.7515312,-95.5903642],
[29.7469666,-95.413642],
[29.7019788,-95.5419795],
[29.7118784,-95.4970781],
[29.7143069,-95.4190756],
[29.6774603,-95.5657805],
[29.6827427,-95.5262247],
[29.650939,-95.4980373],
[29.6355929,-95.4734624],
[30.0585306,-95.2413034],
[29.8650576,-95.4574048],
[29.8588049,-95.4173348],
[29.8415472,-95.3758639],
[29.8818703,-95.3297437],
[29.8263243,-95.3287154],
[29.795349,-95.380751],
[29.8079298,-95.2638717],
[29.7623224,-95.3274328],
[29.7774907,-95.2934356],
[29.7729723,-95.2597734],
[29.7575272,-95.3668266],
[29.735638,-95.3416359],
[29.7289782,-95.3626455],
[29.6952297,-95.3625073],
[29.7103713,-95.3310235],
[29.7070954,-95.2949683],
[29.6682788,-95.3512867],
[29.623533,-95.3745097],
[29.6039379,-95.2377624],
[29.5837969,-95.1430025],
[29.7314782,-95.4441244],
[29.7227783,-95.3160188]]

In [60]:
neighborhoods.loc[(neighborhoods.Longitude + 95).abs() > 1]

,Neighborhood,Latitude,Longitude
19,Eldridge,29.468021,-96.355244
22,Westchase,28.055017,-82.609820
24,Greater Uptown,29.935760,-90.133407
74,Fourth Ward,29.812292,-85.305565
88,Sunnyside,29.139967,-98.052506


In [61]:
neighborhoods.loc[(neighborhoods.Longitude + 95).abs() > 1, ['Latitude', 'Longitude']] = [[29.742661,-95.720858],
[29.7260909,-95.5744971],
[29.7486921,-95.480928],
[29.7568289,-95.3841914],
[29.6641192,-95.3767489]]

In [62]:
dict_cities['Houston'] = neighborhoods

In [85]:
dict_cities['Houston'].loc[dict_cities['Houston'].duplicated(subset=['Latitude', 'Longitude'])]

,Neighborhood,Latitude,Longitude
30,Montrose,29.746967,-95.413642
49,Houston,29.758938,-95.367697
52,Lake Houston,29.758938,-95.367697
59,Houston Gardens,29.758938,-95.367697
84,South Union,29.695230,-95.362507
102,Ellington,29.603938,-95.237762
108,Spring Branch East,29.778449,-95.483190


### Chicago

In [63]:
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago', header=0)[0]

In [64]:
table.head()

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [65]:
neighborhoods = table[['Neighborhood']]

In [66]:
neighborhoods[['Latitude', 'Longitude']] = pd.DataFrame((neighborhoods.Neighborhood + ', Chicago IL, USA').apply(get_coord).tolist(), index=neighborhoods.index)

The geograpical coordinate of Albany Park, Chicago IL, USA are 41.97, -87.72.
The geograpical coordinate of Altgeld Gardens, Chicago IL, USA are 41.65, -87.60.
The geograpical coordinate of Andersonville, Chicago IL, USA are 41.98, -87.67.
The geograpical coordinate of Archer Heights, Chicago IL, USA are 41.81, -87.73.
The geograpical coordinate of Armour Square, Chicago IL, USA are 41.84, -87.63.
The geograpical coordinate of Ashburn, Chicago IL, USA are 41.75, -87.71.
The geograpical coordinate of Auburn Gresham, Chicago IL, USA are 41.74, -87.66.
The geograpical coordinate of Avalon Park, Chicago IL, USA are 41.75, -87.59.
The geograpical coordinate of Avondale, Chicago IL, USA are 41.94, -87.71.
The geograpical coordinate of Avondale Gardens, Chicago IL, USA are 41.94, -87.71.
The geograpical coordinate of Back of the Yards, Chicago IL, USA are 41.81, -87.67.
The geograpical coordinate of Belmont Central, Chicago IL, USA are 41.94, -87.65.
The geograpical coordinate of Belmont Gard

In [67]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0]

,Neighborhood,Latitude,Longitude
6,Ashburn Estates,NaN,NaN
42,Cottage Grove Heights,NaN,NaN
68,Fulton River District,NaN,NaN
95,Irving Woods,NaN,NaN
98,Jackson Park Highlands,NaN,NaN
110,Lakewood / Balmoral,NaN,NaN
112,Legends South (Robert Taylor Homes),NaN,NaN
117,Little Italy,NaN,NaN
118,Little Village,NaN,NaN
157,Old Town,NaN,NaN


Polish Downtown is no longer a neighborhood;<br>
Sheridan Station Corridor cannot be found on Google;<br>
Talley's Corner cannot be found on Google

In [73]:
neighborhoods = neighborhoods.loc[~neighborhoods.Neighborhood.isin(['Polish Downtown', 'Sheridan Station Corridor', "Talley's Corner"])]

In [74]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0]

,Neighborhood,Latitude,Longitude
6,Ashburn Estates,NaN,NaN
42,Cottage Grove Heights,NaN,NaN
68,Fulton River District,NaN,NaN
95,Irving Woods,NaN,NaN
98,Jackson Park Highlands,NaN,NaN
110,Lakewood / Balmoral,NaN,NaN
112,Legends South (Robert Taylor Homes),NaN,NaN
117,Little Italy,NaN,NaN
118,Little Village,NaN,NaN
157,Old Town,NaN,NaN


In [75]:
neighborhoods.loc[neighborhoods.isna().sum(1) > 0, ['Latitude', 'Longitude']] = [[41.7479953,-87.7246529],
[41.7185428,-87.6020408],
[41.8898792,-87.6480075],
[41.9520932,-87.8390223],
[41.769787,-87.5832518],
[41.9799505,-87.6671326],
[41.8091494,-87.6460559],
[41.8675945,-87.6643539],
[41.845231,-87.7209646],
[41.9111221,-87.6492029],
[41.9111221,-87.6492029],
[41.9504386,-87.687881],
[41.921283,-87.663281],
[41.9393819,-87.7164201],
[41.9289822,-87.6605451]]

In [77]:
neighborhoods.loc[(neighborhoods.Latitude - 42).abs() > 1]

,Neighborhood,Latitude,Longitude
176,Ranch Triangle,45.785819,-91.873151
182,River's Edge,47.522525,-111.274075


In [78]:
neighborhoods.loc[(neighborhoods.Latitude - 42).abs() > 1, ['Latitude', 'Longitude']] = [[41.9145052,-87.6606578],
[41.979527, -87.733849]]

In [79]:
neighborhoods.loc[(neighborhoods.Longitude + 87).abs() > 1]

,Neighborhood,Latitude,Longitude
136,New Eastside,42.032973,-88.2488


In [80]:
neighborhoods.loc[(neighborhoods.Longitude + 87).abs() > 1, ['Latitude', 'Longitude']] = [[41.8866036,-87.62147]]

In [81]:
dict_cities['Chicago'] = neighborhoods

In [86]:
dict_cities['Chicago'].loc[dict_cities['Chicago'].duplicated(subset=['Latitude', 'Longitude'])]

,Neighborhood,Latitude,Longitude
10,Avondale Gardens,41.938921,-87.711168
13,Belmont Gardens,41.939796,-87.653328
15,Belmont Terrace,41.939796,-87.653328
17,Beverly View,41.718153,-87.671767
41,Clearing West,41.778677,-87.769202
49,East Beverly,41.718153,-87.671767
50,East Chatham,41.741145,-87.612548
59,Edgewater Beach,41.983369,-87.663952
92,Hyde Park,41.794446,-87.593924
97,Jackowo,41.938921,-87.711168
